In [ ]:
import os
import face_recognition
import numpy as np
import cv2
from tqdm import tqdm
import pickle

class FaceRecognitionSystem:
    def __init__(self, data_dir=None, model_path=None):
        """
        Initialize the face recognition system.

        :param data_dir: Path to the master directory containing subfolders for each person.
        :param model_path: Path to save or load the trained model.
        """
        self.data_dir = data_dir
        self.model_path = model_path
        self.known_face_encodings = []
        self.known_face_names = []

    def load_and_encode_faces(self):
        """
        Load images from the directory structure and encode faces.
        """
        print("Loading and encoding faces...")

        for person in tqdm(os.listdir(self.data_dir)):
            person_dir = os.path.join(self.data_dir, person)
            if not os.path.isdir(person_dir):
                continue

            for img_name in os.listdir(person_dir):
                if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                img_path = os.path.join(person_dir, img_name)

                try:
                    image = face_recognition.load_image_file(img_path)
                    face_encodings = face_recognition.face_encodings(image)

                    if face_encodings:
                        self.known_face_encodings.append(face_encodings[0])
                        self.known_face_names.append(person)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

    def save_model(self):
        """
        Save the trained model to a file.
        """
        if not self.model_path:
            raise ValueError("Model path is not specified.")

        model_data = {
            'encodings': self.known_face_encodings,
            'names': self.known_face_names
        }

        with open(self.model_path, 'wb') as f:
            pickle.dump(model_data, f)

        print(f"Model saved to {self.model_path}")

    def load_model(self):
        """
        Load the trained model from a file.
        """
        if not self.model_path:
            raise ValueError("Model path is not specified.")

        with open(self.model_path, 'rb') as f:
            model_data = pickle.load(f)

        self.known_face_encodings = model_data['encodings']
        self.known_face_names = model_data['names']

        print(f"Model loaded from {self.model_path}")

    def predict_face(self, image_path, tolerance=0.6):
        """
        Predict the identity of a person in a new image.

        :param image_path: Path to the new image.
        :param tolerance: Distance threshold for face matching. Lower is stricter.
        :return: Predicted person name or "Unknown".
        """
        try:
            unknown_image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(unknown_image)

            if not face_encodings:
                return "No face detected"

            for face_encoding in face_encodings:
                distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                best_match_index = np.argmin(distances)

                if distances[best_match_index] <= tolerance:
                    return self.known_face_names[best_match_index]

            return "Unknown"

        except Exception as e:
            print(f"Error processing image at {image_path}: {e}")
            return None

    def run_complete_pipeline(self, new_image_path=None):
        """
        Run the complete pipeline from data loading to prediction.

        :param new_image_path: Path to the new image for prediction.
        """
        if self.data_dir:
            self.load_and_encode_faces()
            if self.model_path:
                self.save_model()

        if new_image_path:
            prediction = self.predict_face(new_image_path)
            print(f"Predicted person: {prediction}")
            return prediction

        return None

if __name__ == "__main__":
    data_dir = r"C:\Users\sande\Desktop\final_year_project\separated_images"
    model_path = r"C:\Users\sande\Desktop\final_year_project\face_recognition_model.pkl"
    test_image_path = r"C:\Users\sande\Desktop\final_year_project\Screenshot 2025-02-10 164403.png"

    face_recognition_system = FaceRecognitionSystem(data_dir=data_dir, model_path=model_path)
    face_recognition_system.run_complete_pipeline(new_image_path=test_image_path)

    # Later, you can load the model and predict using the following code:
    # face_recognition_system = FaceRecognitionSystem(model_path=model_path)
    # face_recognition_system.load_model()
    # prediction = face_recognition_system.predict_face(test_image_path)
    # print(f"Predicted person: {prediction}")

Loading and encoding faces...


 18%|███████████████                                                                    | 2/11 [02:32<11:56, 79.59s/it]

#TESTING

In [7]:
model_path = r"C:\Users\sande\Desktop\final_year_project\face_recognition_model.pkl"
test_image_path=r"C:\Users\sande\Pictures\Screenshots\Screenshot 2025-02-28 205753.png"
face_recognition_system = FaceRecognitionSystem(model_path=model_path)
face_recognition_system.load_model()
prediction = face_recognition_system.predict_face(test_image_path)
print(f"Predicted person: {prediction}")

Model loaded from C:\Users\sande\Desktop\final_year_project\face_recognition_model.pkl
Predicted person: RUP


In [16]:
import os
import cv2
import pickle
import face_recognition

def predict_and_display_images(model_path, test_image_folder, tolerance=0.6):
    """
    Predict the identity of a person in images from a folder and display the results.

    :param model_path: Path to the trained model file.
    :param test_image_folder: Path to the folder containing test images.
    :param tolerance: Distance threshold for face matching. Lower is stricter.
    """
    # Load the trained model
    with open(model_path, 'rb') as f:
        model_data = pickle.load(f)
    
    known_face_encodings = model_data['encodings']
    known_face_names = model_data['names']

    # Iterate through all images in the test folder
    for img_name in os.listdir(test_image_folder):
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue  # Skip non-image files

        img_path = os.path.join(test_image_folder, img_name)
        print(f"Processing image: {img_name}")

        try:
            # Load the image
            image = face_recognition.load_image_file(img_path)
            face_locations = face_recognition.face_locations(image)
            face_encodings = face_recognition.face_encodings(image, face_locations)

            if not face_encodings:
                print(f"No face detected in {img_name}")
                continue

            # Convert the image to BGR format for OpenCV display
            image_display = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Loop through each face found in the image
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Compare the face with known faces
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(distances)

                if distances[best_match_index] <= tolerance:
                    name = known_face_names[best_match_index]
                else:
                    name = "Unknown"

                # Draw a rectangle around the face
                cv2.rectangle(image_display, (left, top), (right, bottom), (0, 255, 0), 2)

                # Draw the predicted name below the face
                cv2.putText(image_display, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

                print(f"Predicted person in {img_name}: {name}")

            # Display the image with predictions
            cv2.imshow("Predicted Image", image_display)
            cv2.waitKey(0)  # Wait for a key press to move to the next image
            cv2.destroyAllWindows()

        except Exception as e:
            print(f"Error processing image {img_name}: {e}")

# Example usage
model_path = r"C:\Users\sande\Desktop\final_year_project\face_recognition_model.pkl"
test_image_folder = r"C:\Users\sande\Desktop\final_year_project\test_image folder"
predict_and_display_images(model_path, test_image_folder)

Processing image: 18866369661.jpg
Predicted person in 18866369661.jpg: SANDY
Processing image: Screenshot 2025-02-10 164137.png
Predicted person in Screenshot 2025-02-10 164137.png: RUP
Processing image: Screenshot 2025-02-10 164403.png
Predicted person in Screenshot 2025-02-10 164403.png: DEEPMALA
Processing image: Screenshot 2025-02-28 204928.png
Predicted person in Screenshot 2025-02-28 204928.png: HARDIK
Processing image: Screenshot 2025-02-28 210740.png
Predicted person in Screenshot 2025-02-28 210740.png: DEEP
Processing image: Screenshot 2025-02-28 211010.png
Predicted person in Screenshot 2025-02-28 211010.png: KRISH
Processing image: Screenshot 2025-02-28 211139.png
Predicted person in Screenshot 2025-02-28 211139.png: ALEX
Processing image: Screenshot 2025-02-28 211502.png
Predicted person in Screenshot 2025-02-28 211502.png: Rounak
Processing image: Screenshot 2025-02-28 215034.png
Predicted person in Screenshot 2025-02-28 215034.png: HRITTIKA
Processing image: Screenshot 20

In [1]:
import os
import cv2
import pickle
import face_recognition
import matplotlib.pyplot as plt

def predict_and_display_images(model_path, test_image_folder, tolerance=0.6):
    """
    Predict the identity of a person in images from a folder and display the results in the notebook.

    :param model_path: Path to the trained model file.
    :param test_image_folder: Path to the folder containing test images.
    :param tolerance: Distance threshold for face matching. Lower is stricter.
    """
    # Load the trained model
    with open(model_path, 'rb') as f:
        model_data = pickle.load(f)
    
    known_face_encodings = model_data['encodings']
    known_face_names = model_data['names']

    # Iterate through all images in the test folder
    for img_name in os.listdir(test_image_folder):
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue  # Skip non-image files

        img_path = os.path.join(test_image_folder, img_name)
        print(f"Processing image: {img_name}")

        try:
            # Load the image
            image = face_recognition.load_image_file(img_path)
            face_locations = face_recognition.face_locations(image)
            face_encodings = face_recognition.face_encodings(image, face_locations)

            if not face_encodings:
                print(f"No face detected in {img_name}")
                continue

            # Convert the image to BGR format for OpenCV display
            image_display = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Loop through each face found in the image
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Compare the face with known faces
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(distances)

                if distances[best_match_index] <= tolerance:
                    name = known_face_names[best_match_index]
                else:
                    name = "Unknown"

                # Draw a rectangle around the face
                cv2.rectangle(image_display, (left, top), (right, bottom), (0, 255, 0), 2)

                # Draw the predicted name below the face
                cv2.putText(image_display, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

                print(f"Predicted person in {img_name}: {name}")

            # Display the image with predictions in the notebook
            plt.figure(figsize=(10, 10))
            plt.imshow(cv2.cvtColor(image_display, cv2.COLOR_BGR2RGB))
            plt.axis('off')  # Hide axes
            plt.title(f"Predicted Image: {img_name}")
            plt.show()

        except Exception as e:
            print(f"Error processing image {img_name}: {e}")

# Example usage
model_path = r"C:\Users\sande\Desktop\final_year_project\face_recognition_model.pkl"
test_image_folder = r"C:\Users\sande\Desktop\final_year_project\test_image folder"
predict_and_display_images(model_path, test_image_folder)

Processing image: 18866369661.jpg
Error processing image 18866369661.jpg: name 'np' is not defined
Processing image: football.jpg
No face detected in football.jpg
Processing image: Screenshot 2025-02-10 164137.png
Error processing image Screenshot 2025-02-10 164137.png: name 'np' is not defined
Processing image: Screenshot 2025-02-10 164403.png
Error processing image Screenshot 2025-02-10 164403.png: name 'np' is not defined
Processing image: Screenshot 2025-02-28 204928.png
Error processing image Screenshot 2025-02-28 204928.png: name 'np' is not defined
Processing image: Screenshot 2025-02-28 210740.png
Error processing image Screenshot 2025-02-28 210740.png: name 'np' is not defined
Processing image: Screenshot 2025-02-28 211010.png
Error processing image Screenshot 2025-02-28 211010.png: name 'np' is not defined
Processing image: Screenshot 2025-02-28 211139.png
Error processing image Screenshot 2025-02-28 211139.png: name 'np' is not defined
Processing image: Screenshot 2025-02-28